In [26]:
import sys
import requests
import os
from pathlib import Path
import boto3
from random import randint, random
from database import Persistence
from multiprocessing import Pool
from datetime import datetime
from statistics import mean

base_api = 'https://api.goverlytics.com/v1'
headers = {'x-api-key':'d41d8cd98f00b204e9800998ecf8427e'}

# Not sure how I should query the data atm, but I'll stick with the API for now
# refer to graphs workspace iypnb for helper functions/code
def grab_data(state, dataset, country, limit):
    if state:
        query_string = f'{base_api}/{dataset}/{country}/{state}?limit={str(limit)}'
    else:
        query_string = f'{base_api}/{dataset}/{country}?limit={str(limit)}'
    return requests.get(query_string, headers=headers).json()['data']

def grab_bills_sponsored(legislation_lod, legislator):
    num_sponsored = 0
    percentage = 0
    for legislation in legislation_lod:
        if legislation['principal_sponsor']==legislator or legislator in legislation['sponsors'] or legislator in legislation['cosponsors']:
            num_sponsored += 1
    percentage = num_sponsored / len(legislation_lod) * 100
    return {
        'bills_sponsored_num': num_sponsored,
        'bills_sponsored_percentage': percentage
    }


In [2]:
state_lst = [x.lower() for x in os.listdir("./scrapers/us/us-states") if len(x) == 2]

In [65]:
rand_state = state_lst[randint(0, len(state_lst)-1)]
# rand_state = 'tn'
print(f'Grabbing data for {rand_state.upper()}')
legislator_data = grab_data(rand_state, 'division-legislators', 'us', 1000)
print(f'Number of datapoints: {len(legislator_data)}')

def get_bday_and_activity(legislator_data):
    bday_lst, years_active_lst = [], []
    for legislator in legislator_data:
        birthday = legislator['birthday']
        years_active = legislator['years_active']
        if years_active:
            years_active = len(legislator['years_active'])
            years_active_lst.append(years_active)
        if birthday:
            bday_lst.append(datetime.today().year - int(birthday.split('-')[0]))
        print(f'\nBirthday: {birthday}\nYears Active: {years_active}')
    return bday_lst, years_active_lst

get_bday_and_activity(legislator_data)

Grabbing data for WA
Number of datapoints: 147

Birthday: None
Years Active: 1

Birthday: None
Years Active: 1

Birthday: None
Years Active: 1

Birthday: None
Years Active: 8

Birthday: None
Years Active: 1

Birthday: None
Years Active: 9

Birthday: None
Years Active: 1

Birthday: None
Years Active: 3

Birthday: None
Years Active: 7

Birthday: None
Years Active: 3

Birthday: None
Years Active: 3

Birthday: None
Years Active: 1

Birthday: None
Years Active: 6

Birthday: None
Years Active: 23

Birthday: None
Years Active: 5

Birthday: None
Years Active: 1

Birthday: None
Years Active: 24

Birthday: None
Years Active: 24

Birthday: None
Years Active: 3

Birthday: None
Years Active: 3

Birthday: None
Years Active: 7

Birthday: None
Years Active: 5

Birthday: None
Years Active: 3

Birthday: None
Years Active: 7

Birthday: None
Years Active: 3

Birthday: None
Years Active: 5

Birthday: None
Years Active: 9

Birthday: None
Years Active: 12

Birthday: None
Years Active: 6

Birthday: None
Years

([53,
  54,
  5,
  77,
  72,
  52,
  52,
  8,
  51,
  8,
  82,
  79,
  74,
  46,
  40,
  71,
  9,
  38,
  10,
  53,
  1,
  54,
  45,
  64,
  59,
  51,
  74,
  47],
 [1,
  1,
  1,
  8,
  1,
  9,
  1,
  3,
  7,
  3,
  3,
  1,
  6,
  23,
  5,
  1,
  24,
  24,
  3,
  3,
  7,
  5,
  3,
  7,
  3,
  5,
  9,
  12,
  6,
  3,
  1,
  3,
  15,
  3,
  7,
  1,
  11,
  11,
  8,
  1,
  3,
  11,
  2,
  21,
  1,
  13,
  5,
  1,
  5,
  17,
  3,
  3,
  14,
  9,
  5,
  5,
  7,
  1,
  3,
  5,
  20,
  19,
  8,
  13,
  3,
  7,
  11,
  2,
  3,
  9,
  2,
  3,
  1,
  11,
  23,
  15,
  17,
  9,
  3,
  1,
  17,
  5,
  7,
  7,
  17,
  3,
  5,
  1,
  3,
  11,
  5,
  9,
  5,
  3,
  5,
  1,
  11,
  11,
  3,
  8,
  8,
  8,
  6,
  8,
  8,
  10,
  8,
  2,
  4,
  10,
  4,
  10,
  1,
  8,
  12,
  2,
  12,
  4,
  12,
  12,
  4,
  4,
  8,
  2,
  1,
  8,
  2,
  2,
  1,
  10,
  8,
  2,
  1,
  10,
  4,
  2,
  12,
  4,
  2,
  6,
  4,
  12,
  4,
  4,
  2,
  1,
  4])

In [34]:
len(bday_lst), len(years_active_lst)

(123, 106)

In [19]:
# rand_state = state_lst[randint(0, len(state_lst)-1)]
# print(f'grabbing data for {rand_state}')

def get_ave_bills_sponsored(state):
    try:
        legislation_requests = grab_data(state, 'division-legislation', 'us', 2000)
        legislator_requests = grab_data(state, 'division-legislators', 'us', 1000)

        # gets average bills sponsored for legislator in state
        data = [grab_bills_sponsored(legislation_requests, x['name_last']) for x in legislator_requests]
        ave_percentage_data = sum([x['bills_sponsored_percentage'] for x in data])/ len(data)
        ave_percentage_data = round(ave_percentage_data, 2)
        ave_num_data = sum([x['bills_sponsored_num'] for x in data])/ len(data)
        ave_num_data = round(ave_num_data, 2)

        print(f'Average bills sponsored per legislator in {state.upper()}: {ave_num_data}')
        print(f'As a percentage: {ave_percentage_data}')
        return {
            'state_name':state,
            'ave_num_data':ave_num_data,
            'ave_percentage_data':ave_percentage_data
        }

    except KeyError:
        print(f'KeyError for state {state.upper()}')
        return ''

info_dict = []
for state in state_lst[1:6]:
    info_dict.append(get_ave_bills_sponsored(state))
[x for x in info_dict if x]

Average bills sponsored per legislator in AL: 2.52
As a percentage: 0.15
Average bills sponsored per legislator in AR: 4.33
As a percentage: 0.25
Average bills sponsored per legislator in AZ: 80.26
As a percentage: 4.4
KeyError for state CO
Average bills sponsored per legislator in DE: 61.08
As a percentage: 19.09


[{'state': 'al', 'ave_num_data': 2.52, 'ave_percentage_data': 0.15},
 {'state': 'ar', 'ave_num_data': 4.33, 'ave_percentage_data': 0.25},
 {'state': 'az', 'ave_num_data': 80.26, 'ave_percentage_data': 4.4},
 {'state': 'de', 'ave_num_data': 61.08, 'ave_percentage_data': 19.09}]

In [60]:
country, state, limit, year = 'us', 'ar', 1000, 2021

query_string = f'{base_api}/division-legislators/{country}/{state}?limit={limit}'
data = requests.get(query_string, headers=headers).json()['data']


In [62]:
current_year = datetime.today().year
legislator_lst = []
for legislator in data:
    most_recent_term_id = legislator['most_recent_term_id']
    years_active = legislator['years_active']
    if most_recent_term_id == current_year or current_year in years_active:
        legislator_lst.append(legislator)
len(legislator_lst)

670

In [64]:
legislator_lst[-1]

{'goverlytics_id': 30570,
 'source_id': '',
 'most_recent_term_id': '1997',
 'date_collected': '2021-04-30T12:37:31.620391',
 'source_url': 'https://www.arkleg.state.ar.us/Legislators/Detail?member=Young&ddBienniumSession=1997%2FR',
 'name_full': 'Dennis Young',
 'name_last': 'Young',
 'name_first': 'Dennis',
 'name_middle': '',
 'name_suffix': '',
 'country_id': 1,
 'country': 'United States of America',
 'state_id': 5,
 'state': 'AR',
 'party_id': None,
 'party': '',
 'role': 'Representative',
 'district': '21',
 'years_active': [1979,
  1980,
  1986,
  1987,
  1988,
  1989,
  1990,
  1991,
  1992,
  1993,
  1994,
  1995,
  1996,
  1997,
  1998,
  1999,
  2000,
  2001,
  2002,
  2003,
  2004,
  2005,
  2006,
  2007,
  2008,
  2009,
  2010,
  2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2020,
  2021],
 'committees': [],
 'areas_served': [],
 'phone_numbers': [{'district office': '870-774-3669'}],
 'addresses': [{'address': 'P. O. Box 1835, Texarkana, 75504',